<a href="https://colab.research.google.com/github/Snerdify/ML-/blob/main/ColBERT_BEIR_Vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U datasets -qq

In [ ]:
from datasets import load_dataset, Dataset

corpus, queries = load_dataset("BeIR/trec-covid", "corpus")["corpus"], load_dataset("BeIR/trec-covid", "queries")["queries"]
corpus = corpus.map(lambda x:{"combined_text": f"{x['title']}\n{x['text']}"})

In [ ]:
corpus = corpus.select(range(2048))
all_passages = corpus["combined_text"]

In [ ]:
!pip install git+https://github.com/stanford-futuredata/ColBERT -qq

  Preparing metadata (setup.py) ... done


In [ ]:
from colbert.modeling.checkpoint import Checkpoint
from colbert.infra import ColBERTConfig
import pandas
import numpy
import torch

In [ ]:
ckpt = Checkpoint("colbert-ir/colbertv2.0", colbert_config=ColBERTConfig(
            root="experiments",
        ))

In [ ]:
# queries['text']

In [ ]:
%%time
Q = ckpt.queryFromText(queries['text'], bsize=64)
Q.shape

CPU times: user 33.1 ms, sys: 2.83 ms, total: 35.9 ms
Wall time: 21.9 ms


torch.Size([50, 32, 128])

In [ ]:
torch.save(Q, "queries.pt")

In [ ]:
D = ckpt.docFromText(all_passages, bsize=1024)[0]

In [ ]:
D.shape

torch.Size([2048, 180, 128])

In [ ]:
torch.save(D, "documents-2K.pt")